<p style="text-align:center;">
  <img src="Termo_Aplicada_P2_13082025-fig-a.png"
       alt="Figura — Termodinâmica Aplicada, P2"
       style="width:80%; height:auto; display:block; margin:0 auto;">
</p>


# Utilitários: Propriedades Termodinâmicas e Eficiências Isentrópicas

Esta seção documenta as funções auxiliares desta célula para (i) obter propriedades via **CoolProp** e (ii) calcular estados de saída de **turbinas** e **compressores** com eficiência isentrópica. Inclui também relações de gás ideal para ar (modelo ar-padrão).

**Dependências e unidades (SI, PropsSI):** \(T\) em K, \(p\) em Pa, \(h\) em J/kg, \(s\) em J/(kg·K). Se **CoolProp** não estiver disponível, use tabelas/valores manuais.

---

## Turbina — eficiência isentrópica e entalpia de saída

Definição da eficiência isentrópica da turbina:
$$
\eta_t \;=\; \frac{h_1 - h_{2,\text{real}}}{h_1 - h_{2s}},
$$
onde $h_{2s} = h(p_2, s_1)$ é a entalpia de saída **isentrópica** (mesma entropia da entrada, pressão de saída).

Rearranjo para a entalpia real de saída:
$$
h_{2,\text{real}} \;=\; h_1 \;-\; \eta_t\,\bigl(h_1 - h_{2s}\bigr).
$$

---

## Compressor — eficiência isentrópica e entalpia de saída

Definição da eficiência isentrópica do compressor:
$$
\eta_c \;=\; \frac{h_{2s} - h_1}{h_{2,\text{real}} - h_1}.
$$

Rearranjo:
$$
h_{2,\text{real}} \;=\; h_1 \;+\; \frac{h_{2s} - h_1}{\eta_c}.
$$

---

## Relações de gás ideal para o ar (modelo ar-padrão)

Defina a razão de pressões ($r_p = p_2/p_1$) e o expoente de expansão isentrópica ($\gamma$).

Transformação isentrópica ideal:
$$
T_{2s} \;=\; T_1 \, r_p^{(\gamma-1)/\gamma}.
$$

Compressor com eficiência em temperaturas:
$$
\eta_c \;=\; \frac{T_{2s} - T_1}{T_2 - T_1}
\;\;\Longrightarrow\;\;
T_2 \;=\; T_1 \;+\; \frac{T_{2s} - T_1}{\eta_c}.
$$



In [11]:

# Utilities (optional)
# - Tries to import CoolProp for water and R134a properties.
# - Helper functions for turbine/compressor isentropic efficiency.
try:
    from CoolProp.CoolProp import PropsSI
    HAS_COOLPROP = True
except Exception as e:
    HAS_COOLPROP = False
    print("CoolProp not found. Use tables or fill values manually. Error:", e)

def turbine_h_out(h_in, s_in, p_out, eta_is=1.0, fluid='Water'):
    # Turbine: eta_t = (h_in - h_out_real)/(h_in - h_out_is)
    if not HAS_COOLPROP:
        raise RuntimeError("CoolProp not available.")
    h2s = PropsSI('H','S',s_in,'P',p_out,fluid)
    h2  = h_in - eta_is*(h_in - h2s)
    return h2, {'h2s': h2s, 'h2': h2}

def compressor_h_out(h_in, s_in, p_out, eta_is=1.0, fluid='Air'):
    # Compressor: eta_c = (h_out_is - h_in)/(h_out_real - h_in)
    if not HAS_COOLPROP:
        raise RuntimeError("CoolProp not available.")
    h2s = PropsSI('H','S',s_in,'P',p_out,fluid)
    if eta_is <= 0:
        raise ValueError("eta_is must be > 0")
    h2  = h_in + (h2s - h_in)/eta_is
    return h2, {'h2s': h2s, 'h2': h2}

def T2_isentropic_air(T1, r_p, gamma=1.4):
    # Ideal-gas (air-standard): T2s = T1 * r_p^((gamma-1)/gamma)
    return T1 * (r_p**((gamma-1.0)/gamma))

def T2_with_eta_comp_air(T1, r_p, eta_c=1.0, gamma=1.4):
    # eta_c = (T2s - T1)/(T2 - T1)  =>  T2 = T1 + (T2s - T1)/eta_c
    T2s = T2_isentropic_air(T1, r_p, gamma=gamma)
    return T1 + (T2s - T1)/eta_c, T2s



## 1) Ciclo Rankine com Reaquecimento — **4,0 pontos**

**1ª Questão)** Considere um Ciclo Rankine com Reaquecimento, operando nas pressões 10, 700 e 6000 kPa, com todas as máquinas de fluxo (i.g., bombas, turbinas) com eficiência isentrópica de 80%. Nas duas saídas da caldeira, a temperatura é 360 °C. Calcule o título na saída da turbina de alta pressão. (4,0 pontos)

### Hipóteses
- Regime permanente
- Caldeira e condensador isobáricos
- Bomba e turbinas adiabáticas
- Quedas de pressão e perdas de energia são desprezados nas conexões 
- Caldeira fornece vapor **superaquecido** a 6 MPa e 400 °C.
- Reaquecimento eleva novamente a **400 °C** a 0,7 MPa.
- η_turbinas = η_bombas = **0,80** (definição de eficiência isentrópica).
- Condensador a **10 kPa**.


<p style="text-align:center;">
  <img src="Termo_Aplicada_P2_13082025-fig-b.png"
       alt="Figura — Termodinâmica Aplicada, P2"
       style="width:80%; height:auto; display:block; margin:0 auto;">
</p>



**Objetivo.** Determinar o estado após a turbina de alta, calculando $h_2$, identificando a fase em $p_2$ e, se aplicável, o título $x$.

**Dados.** $\eta_t=0{,}80$, $p_1=6{,}0\,\text{MPa}$, $p_2=0{,}8\,\text{MPa}$, $T_{\mathrm{boiler\_out}}=360^\circ\text{C}$. Propriedades em SI: $T$ [K], $p$ [Pa], $h$ [J/kg], $s$ [J/(kg·K)].

---

## Passos e equações

1) **Estado 1 (entrada da turbina).**  
$$
h_1 = h(p_1,\,T_{\mathrm{boiler\_out}}), \qquad
s_1 = s(p_1,\,T_{\mathrm{boiler\_out}})
$$

2) **Saída isentrópica (2s).**  
$$
h_{2s} = h(p_2,\,s_1)
$$

3) **Eficiência isentrópica da turbina.**  
$$
\eta_t = \frac{h_1 - h_{2,\mathrm{real}}}{h_1 - h_{2s}}
\;\;\Rightarrow\;\;
h_2 \equiv h_{2,\mathrm{real}} = h_1 - \eta_t\,(h_1 - h_{2s})
$$

4) **Verificação de fase em $p_2$.**  
$$
h_f = h(p_2,\,x{=}0), \qquad h_g = h(p_2,\,x{=}1)
$$
- Se $h_f \le h_2 \le h_g$: mistura bifásica, com
$$
x = \frac{h_2 - h_f}{\,h_g - h_f\,}
$$
- Se $h_2 < h_f$: líquido comprimido.  
- Se $h_2 > h_g$: vapor superaquecido.

In [12]:

# === Q1: Estado após a turbina de alta (6 MPa -> 0,8 MPa) ===
eta_t = 0.80
p_hi   = 6_000_000.0       # Pa (6 MPa)
p_reh  =   700_000.0       # Pa (0.8 MPa)
T_boiler_out  = 360.0 + 273.15  # K

if HAS_COOLPROP:
    fluid = 'Water'
    # Estado 1: entrada turbina de alta
    h1 = PropsSI('H','P',p_hi,'T',T_boiler_out,fluid)
    s1 = PropsSI('S','P',p_hi,'T',T_boiler_out,fluid)
    # Expansão 1 -> 2 (turbina de alta): 6 MPa -> 0.8 MPa
    h2, aux = turbine_h_out(h1, s1, p_reh, eta_is=eta_t, fluid=fluid)

    # Verificar fase em (p=0.8 MPa, h=h2)
    Tsat_reh = PropsSI('T','P',p_reh,'Q',0,fluid)
    hf = PropsSI('H','P',p_reh,'Q',0,fluid)
    hg = PropsSI('H','P',p_reh,'Q',1,fluid)
    if hf <= h2 <= hg:
        x = (h2 - hf)/(hg - hf)
        fase = "bifásica"
    else:
        x = None
        fase = "liquido comprimido" if h2 < hf else "vapor superaquecido"

    print(f"Estado 1 (6 MPa, 360 C): h1={h1/1000:.2f} kJ/kg, s1={s1/1000:.3f} kJ/(kg.K)")
    print(f"Estado 2 (0.7 MPa, apos turbina de alta): h2={h2/1000:.2f} kJ/kg  -> fase: {fase}")
    if x is not None:
        print(f"Titulo na saida da turbina de alta: x = {x:.4f}")
    else:
        print("Titulo nao aplicavel (estado nao esta na cupula de saturacao).")
else:
    print("CoolProp indisponivel. Use tabelas de vapor:")
    print("1) Obter h1, s1 em (6 MPa, 360 C).")
    print("2) Em p=0.8 MPa, encontrar h2s com s=s1 (superaquecido).")
    print("3) h2 = h1 - eta_t*(h1 - h2s).")
    print("4) Verificar fase em p=0.8 MPa e calcular x se bifasico.")


Estado 1 (6 MPa, 360 C): h1=3071.99 kJ/kg, s1=6.380 kJ/(kg.K)
Estado 2 (0.7 MPa, apos turbina de alta): h2=2710.11 kJ/kg  -> fase: bifásica
Titulo na saida da turbina de alta: x = 0.9745


## 2) Ciclo de Refrigeração por Compressão a Vapor (R134a) — **3,0 pontos**

**2ª Questão)**  O dispostivo de expansão de um Ciclo de Refrigeração por Compressão a Vapor (R134a) opera entre as temperaturas 40 °C e 5 °C. Qual o COP (i.e. coeficiente de perfomance, coeficiente de desempenho) do ciclo?  (3,0 pontos)

### Hipóteses
- Regime permanente
- Compressor adiabático e isentrópico
- **Ciclo ideal** (compressão isentrópica), **sem superaquecimento** na saída do evaporador e **sem sub-resfriamento** na saída do condensador.
- Evaporação a **4 °C (saturado)** e condensação a **40 °C (saturado)**.
- Válvula de expansão **isentálpica**, sem variação de energia cinética e potencial.
- Desprezar perdas de pressão em trocadores/linhas.  


<p style="text-align:center;">
  <img src="Termo_Aplicada_P2_13082025-fig-c.png"
       alt="Figura — Termodinâmica Aplicada, P2"
       style="width:80%; height:auto; display:block; margin:0 auto;">
</p>

**Objetivo.** Calcular o coeficiente de desempenho (COP) de um ciclo ideal de compressão de vapor com R134a operando entre $T_e=4\,^\circ\mathrm{C}$ e $T_c=40\,^\circ\mathrm{C}$.

**Hipóteses.** Compressor isentrópico, condensação e evaporação a pressão constante, dispositivo de expansão isentálpico. Propriedades no sistema SI.

---

## Estados e relações termodinâmicas

1) **Entrada do compressor $(1)$ — vapor saturado a $T_e$**
$$
p_1 = p_{\mathrm{sat}}(T_e), \qquad
h_1 = h(T_e, x=1), \qquad
s_1 = s(T_e, x=1)
$$

2) **Saída do compressor ideal $(2)$ — compressão isentrópica até $p_{\mathrm{cond}}$**
$$
p_2 = p_{\mathrm{cond}} = p_{\mathrm{sat}}(T_c) \quad\text{(no código, } p_2 \equiv p_3\text{)}, \qquad
s_{2s} = s_1, \qquad
h_{2s} = h(p_2, s_1)
$$
Como o compressor é ideal, vale $h_2 = h_{2s}$.

3) **Saída do condensador $(3)$ — líquido saturado a $T_c$**
$$
p_3 = p_{\mathrm{sat}}(T_c), \qquad
h_3 = h(T_c, x=0)
$$

4) **Saída da válvula $(4)$ — expansão isentálpica**
$$
h_4 = h_3
$$

---

## Balanços específicos e COP

Calor específico absorvido no evaporador:
$$
q_L = h_1 - h_4
$$

Trabalho específico do compressor:
$$
w_C = h_2 - h_1
$$

Coeficiente de desempenho do refrigerador:
$$
\mathrm{COP}_R = \frac{q_L}{w_C} = \frac{h_1 - h_4}{h_2 - h_1}
$$

---

In [14]:
# === Q2: COP de um ciclo ideal de compressao de vapor com R134a ===
Te = 4.0 + 273.15     # K
Tc = 40.0 + 273.15    # K
fluid = 'R134a'

if HAS_COOLPROP:
    # 1: vapor saturado a Te
    p1 = PropsSI('P','T',Te,'Q',1,fluid)
    h1 = PropsSI('H','T',Te,'Q',1,fluid)
    s1 = PropsSI('S','T',Te,'Q',1,fluid)
    # 3: liquido saturado a Tc
    h3 = PropsSI('H','T',Tc,'Q',0,fluid)
    p3 = PropsSI('P','T',Tc,'Q',0,fluid)
    # 4: valvula isentalpica
    h4 = h3
    # 2s: compressao isentropica ate pressao de saturacao a Tc
    h2s = PropsSI('H','S',s1,'P',p3,fluid)
    h2  = h2s  # compressor ideal base
     

    qL = h1 - h4
    wC = h2 - h1
    COP = qL / wC

    print(f"p_evap = {p1/1000:.1f} kPa, p_cond = {p3/1000:.1f} kPa")
    print(f"h1={h1/1000:.2f} kJ/kg, h2={h2/1000:.2f} kJ/kg, h3={h3/1000:.2f} kJ/kg, h4={h4/1000:.2f} kJ/kg")
    print(f"qL={qL/1000:.2f} kJ/kg, wC={wC/1000:.2f} kJ/kg,  COP={COP:.2f}")
else:
    print("CoolProp indisponivel. Passos com tabelas do R134a:")
    print("1) h1,s1 em Te=4 C (vapor sat.)  2) h2s por s=s1 a p_cond (40 C)")
    print("3) h3 em Tc=40 C (liquido sat.)  4) h4=h3  -> COP=(h1-h4)/(h2-h1)")


p_evap = 337.7 kPa, p_cond = 1016.6 kPa
h1=400.92 kJ/kg, h2=423.81 kJ/kg, h3=256.41 kJ/kg, h4=256.41 kJ/kg
qL=144.51 kJ/kg, wC=22.89 kJ/kg,  COP=6.31



## 3) Ciclo Brayton (motor de avião) — **3,0 pontos**

**3ª Questão)** O motor de um avião (Ciclo Brayton), aspira ar a 25 °C e 100 kPa e tem uma relação de pressão de 10:1. Qual a temperatura na entrada da câmara de combustão? (3,0 pontos)  

### Hipóteses
- **Ar-padrão** com propriedades constantes (γ = 1,4; c_p = 1,004 kJ/(kg·K)).
- Regime permanente
- **Compressão isentrópica** (caso base). Alternativa: adote **η_c** específico e calcule T₂ real.
- Desprezar perdas no duto entre compressor e câmara.

<p style="text-align:center;">
  <img src="Termo_Aplicada_P2_13082025-fig-d.png"
       alt="Figura — Termodinâmica Aplicada, P2"
       style="width:80%; height:auto; display:block; margin:0 auto;">
</p>

**Objetivo.** Determinar as temperaturas de saída do compressor em um modelo ar-padrão:
- caso isentrópico: $T_{2s}$,
- caso com eficiência do compressor: $T_2$ para um dado $\eta_c$.

**Definições e dados.** Temperatura e pressão na entrada: $T_1$, $p_1$; razão de pressões $r_p=p_2/p_1$; expoente adiabático $\gamma$ (ar seco: $\gamma \approx 1{,}4$). As temperaturas estão em K e as pressões são absolutas.

---

## Relações termodinâmicas

1) **Saída isentrópica do compressor (gás ideal):**
$$
T_{2s} \;=\; T_1 \, r_p^{\frac{\gamma-1}{\gamma}} .
$$

2) **Eficiência isentrópica do compressor:**
$$
\eta_c \;=\; \frac{T_{2s}-T_1}{T_2-T_1}
\;\;\Longrightarrow\;\;
T_2 \;=\; T_1 \;+\; \frac{T_{2s}-T_1}{\eta_c} .
$$

**Observações.** Para $\eta_c=1$, tem-se $T_2=T_{2s}$. Para $0<\eta_c<1$, vale $T_2>T_{2s}$.

In [15]:

# === Q3: Temperatura na entrada da camara (saida do compressor) ===
T1 = 25.0 + 273.15   # K
p1 = 100_000.0       # Pa
r_p = 10.0           # p2/p1
gamma = 1.4

# Caso base: compressor isentropico
T2s = T1 * (r_p ** ((gamma - 1.0)/gamma))

# Se quiser considerar eta_c != 1:
eta_c = 1.0   # edite para, por exemplo, 0.85
T2 = T1 + (T2s - T1)/eta_c

print(f"T1 = {T1:.2f} K  (25 C)")
print(f"r_p = {r_p:.2f},  gamma = {gamma:.2f}")
print(f"T2s (isentropico) = {T2s:.2f} K")
print(f"Assumindo eta_c = {eta_c:.2f}  =>  T2 = {T2:.2f} K (entrada da camara de combustao)")


T1 = 298.15 K  (25 C)
r_p = 10.00,  gamma = 1.40
T2s (isentropico) = 575.64 K
Assumindo eta_c = 1.00  =>  T2 = 575.64 K (entrada da camara de combustao)
